# Taxi Fare Prediction

* Adaptado de https://towardsdatascience.com/another-machine-learning-walk-through-and-a-challenge-8fae1e187a64

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

palette = sns.color_palette('Paired', 10)
plt.style.use('seaborn-darkgrid')

RSEED = 42

In [ ]:
data = pd.read_csv('../input/train.csv', nrows = 1_000_000, 
                   parse_dates = ['pickup_datetime']).drop(columns = 'key')

data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.head()

In [ ]:
data.info()

# Identificando Outliers

In [ ]:
data.describe()

### Exploração e Limpeza dos dados

In [ ]:
plt.figure(figsize = (10, 6))
sns.distplot(data['fare_amount'])
plt.title('Distribution of Fare');

In [ ]:
print("Passagens com valor negativo: {}".format(len(data[data['fare_amount'] < 0])))
print("Passagens com valor 0: {}".format(len(data[data['fare_amount'] == 0])))
print("Passagens com valor maior : {}".format(len(data[data['fare_amount'] > 100])))

### Removendo Outliers

Baseado [nesta matéria](http://nymag.com/nymetro/urban/features/taxi/n_20286/), removeremos as passagens com valor menor que $2.5.

* Tambem limitaremos o valor máximo das passagens a $100.0.

In [ ]:
data = data[data['fare_amount'].between(left = 2.5, right = 100)]

Discretizamos as passagens para visualizar suas distribuições

In [ ]:
# Discretiza os valores das passagens em faixas
data['fare-bin'] = pd.cut(data['fare_amount'], bins = list(range(0, 50, 5))).astype(str)
data.loc[data['fare-bin'] == 'nan', 'fare-bin'] = '[45+]'

# Visualizando as faixas de valores
data.loc[data['fare-bin'] == '(5, 10]', 'fare-bin'] = '(05, 10]'
data['fare-bin'].value_counts().sort_index().plot.bar(color = 'b', edgecolor = 'k')
plt.title('Fare Binned');

### Outros Outliers

In [ ]:
data['passenger_count'].value_counts().sort_index().plot.bar(color = 'b', edgecolor = 'k')
plt.title('Passenger Counts')
plt.xlabel('Number of Passengers')
plt.ylabel('Count');

In [ ]:
data = data.loc[data['passenger_count'] <= 6]

Para `latitude` e `longitude`, removeremos os valores abaixo de 2.5% e acima de 97,5% percentil.

In [ ]:
print('Observasões iniciais: {}'.format(data.shape[0]))

In [ ]:
for col in ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']:
    print(f'{col.capitalize():17}: 2.5% = {round(np.percentile(data[col], 2.5), 2):5} \t 97.5% = {round(np.percentile(data[col], 97.5), 2)}')

In [ ]:
# Remover latitude e longtiude outliers
data = data.loc[data['pickup_latitude'].between(40, 41)]
data = data.loc[data['pickup_longitude'].between(-74, -73)]
data = data.loc[data['dropoff_latitude'].between(40, 41)]
data = data.loc[data['dropoff_longitude'].between(-74, -73)]

print('Novas observações: {}'.format(data.shape[0]))

Agora podemos exibir `latitude` e `longitude` para ver a distributioção. Usaremos apenas 1000 observações.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (20, 8), sharex=True, sharey=True)
axes = axes.flatten()

# Plot Longitude (x) e Latitude (y)
sns.regplot('pickup_longitude', 'pickup_latitude', fit_reg = False, 
            data = data.sample(10000, random_state = RSEED), ax = axes[0]);
sns.regplot('dropoff_longitude', 'dropoff_latitude', fit_reg = False, 
            data = data.sample(10000, random_state = RSEED), ax = axes[1]);
axes[0].set_title('Pickup Locations')
axes[1].set_title('Dropoff Locations');

# Visualizando o dataset

Para uma representação mais contextualizada, podemos traçar a coleta e a entrega em cima de um mapa de Nova York

In [ ]:
BB_zoom = (-74.1, -73.7, 40.6, 40.85)
nyc_map_zoom = plt.imread("https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/images/nyc_-74.1_-73.7_40.6_40.85.PNG?raw=true")

Essa função é para plotar os dados no mapa de Nova York

In [ ]:
def plot_on_map(df, BB, nyc_map, s=10, alpha=0.2, color = False):
    fig, axs = plt.subplots(2, 1, figsize=(18, 22))
    axs[0].scatter(df.pickup_longitude, df.pickup_latitude, zorder=1, alpha=alpha, c='r', s=2)
    axs[0].set_xlim((BB[0], BB[1]))
    axs[0].set_ylim((BB[2], BB[3]))
    axs[0].set_title('Localizações de embarque')
    axs[0].axis('off')
    axs[0].imshow(nyc_map, zorder=0, extent=BB)

    axs[1].scatter(df.dropoff_longitude, df.dropoff_latitude, zorder=1, alpha=alpha, c='b', s=2)
    axs[1].set_xlim((BB[0], BB[1]))
    axs[1].set_ylim((BB[2], BB[3]))
    axs[1].set_title('Locais de desembarque')
    axs[1].axis('off')
    axs[1].imshow(nyc_map, zorder=0, extent=BB)

In [ ]:
plot_on_map(data.sample(500_000, random_state = RSEED), 
            BB_zoom, nyc_map_zoom, s=0.05, alpha=0.05)

A maioria dos passeios está concentrada em Manhattan e aparentemente há mais passeios terminando fora de Manhattan do que dentro.

Para verificar se há uma diferença nos locais com base na tarifa, vamos mapear cada compartimento de tarifa para uma cor diferente e, em seguida, plotar o gráfico.

In [ ]:
color_mapping = {fare_bin: palette[i] for i, fare_bin in enumerate(data['fare-bin'].unique())}
color_mapping

A função de mapa substituirá os valores em uma coluna pelos valores correspondentes no dicionário.

In [ ]:
data['color'] = data['fare-bin'].map(color_mapping)
plot_data = data.sample(500_000, random_state = RSEED)

In [ ]:
BB = BB_zoom

fig, axs = plt.subplots(1, 1, figsize=(20, 18))


for b, df in plot_data.groupby('fare-bin'):
    axs.scatter(df.pickup_longitude, df.pickup_latitude, zorder=1, alpha=0.2, c=df.color, s=30, label = f'{b}')
    axs.set_xlim((BB[0], BB[1]))
    axs.set_ylim((BB[2], BB[3]))
    axs.set_title('Localizações de embarque', size = 32)
    axs.axis('off')
    
# Legenda
leg = axs.legend(fontsize = 20, markerscale = 3)

for lh in leg.legendHandles: 
    lh.set_alpha(1)

leg.set_title('Tarifa', prop = {'size': 28})

axs.imshow(nyc_map_zoom, zorder=0, extent=BB_zoom);

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(20, 18))


for b, df in plot_data.groupby('fare-bin'):
    axs.scatter(df.dropoff_longitude, df.dropoff_latitude, zorder=1, 
                alpha=0.2, c=df.color, s=30, label = f'{b}')
    axs.set_xlim((BB[0], BB[1]))
    axs.set_ylim((BB[2], BB[3]))
    axs.set_title('Localizações de desembarque', size = 32)
    axs.axis('off')
    
# Legenda
leg = axs.legend(fontsize = 20, markerscale = 3)

for lh in leg.legendHandles: 
    lh.set_alpha(1)

leg.set_title('Tarifa', prop = {'size': 28})

axs.imshow(nyc_map_zoom, zorder=0, extent=BB_zoom);

Além de interessantes, os gráficos podem nos ajudar a identificar anomalias, relacionamentos ou idéias para novos recursos. 

Os passeios mais caros tendem a se agrupar em torno do aeroporto. Se soubéssemos com certeza que havia passeio com destino ao aeroporto, saberíamos a tarifa!

# Engenharia de Features

### Distância absoluta

In [ ]:
data['abs_lat_diff'] = (data['dropoff_latitude'] - data['pickup_latitude']).abs()
data['abs_lon_diff'] = (data['dropoff_longitude'] - data['pickup_longitude']).abs()

In [ ]:
sns.lmplot('abs_lat_diff', 'abs_lon_diff', fit_reg = False,
           data = data.sample(10000, random_state=RSEED));
plt.title('Absolute latitude difference vs Absolute longitude difference');

Verificamos se existe alguma distância com valor 0.

In [ ]:
data.shape, ((data['abs_lat_diff'] == 0) & (data['abs_lon_diff'] == 0)).sum()

Vamos remover essas observações

In [ ]:
no_diff = (data['abs_lat_diff'] == 0) & (data['abs_lon_diff'] == 0)
data = data.loc[~no_diff]
data.shape

In [ ]:
sns.lmplot('abs_lat_diff', 'abs_lon_diff', hue = 'fare-bin', height = 8, palette=palette,
           fit_reg = False, data = data.sample(10000, random_state=RSEED))
plt.title('Absolute latitude difference vs Absolute longitude difference');

> ### Distâncias de Manhattan e Euclidiana

[Distância de Minkowski](https://en.wikipedia.org/wiki/Minkowski_distance) entre dois pontos:

$${\displaystyle D\left(X,Y\right)=\left(\sum _{i=1}^{n}|x_{i}-y_{i}|^{p}\right)^{1/p}}$$

* se p = 1, temos a distancia de Manhattan, e se p = 2 temos a distancia Euclidiana.

In [ ]:
def minkowski_distance(x1, x2, y1, y2, p):
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)

In [ ]:
print('Minkowski: {} \nEuclidiana: {}'.format(minkowski_distance(0, 3, 0, 4, 1), minkowski_distance(0, 3, 0, 4, 2)))

Aplicaremos estas funções para `latitude` e `longitude`. Lembrando que estas não são as distancias reais, pois estas funções são aplicadas considerando o plano cartesiano.

In [ ]:
data['manhattan'] = minkowski_distance(data['pickup_longitude'], data['dropoff_longitude'],
                                       data['pickup_latitude'], data['dropoff_latitude'], 1)

plt.figure(figsize = (12, 6))

for f, grouped in data.groupby('fare-bin'):
    sns.kdeplot(grouped['manhattan'], label = f'{f}')

plt.xlabel('degrees')
plt.ylabel('density')
plt.title('Manhattan Distance by Fare Amount');

In [ ]:
data.groupby('fare-bin')['manhattan'].agg(['mean', 'count'])

In [ ]:
data['euclidean'] = minkowski_distance(data['pickup_longitude'], data['dropoff_longitude'],
                                       data['pickup_latitude'], data['dropoff_latitude'], 2)

plt.figure(figsize = (12, 6))
for f, grouped in data.groupby('fare-bin'):
    sns.kdeplot(grouped['euclidean'], label = f'{f}')

plt.xlabel('degrees')
plt.ylabel('density')
plt.title('Euclidean Distance by Fare Amount');

In [ ]:
data.groupby('fare-bin')['euclidean'].agg(['mean', 'count'])

### Haversine Distance

[A fórmula de Haversine](https://www.wikiwand.com/pt/F%C3%B3rmula_de_Haversine) é uma importante equação usada em navegação, fornecendo distâncias entre dois pontos de uma esfera a partir de suas latitudes e longitudes. É um caso especial de uma fórmula mais geral de trigonometria esférica, a lei dos Haversines, relacionando os lados a ângulos de uma esfera "triangular".

$${\displaystyle 2R\arcsin \left({\sqrt {\sin ^{2}\left({\frac {lat _{2}-lat _{1}}{2}}\right)+\cos(lat _{1})\cos(lat _{2})\sin ^{2}\left({\frac {lon _{2}-lon _{1}}{2}}\right)}}\right)}$$

[Funções matemáticas do numpy](https://docs.scipy.org/doc/numpy-1.15.1/reference/routines.math.html)

[Funcção implementada](https://stackoverflow.com/a/29546836)


In [ ]:
# Radius of the earth in kilometers
R = 6378

def haversine_distance(lon1, lat1, lon2, lat2):
    # Converte latitude e longitude para radiano
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Calcula diferença entre latitude e longitude
    diffLat = lat2 - lat1
    diffLon = lon2 - lon1
    
    # Aplicando a formula 
    distance = 2 * R * np.arcsin( np.sqrt( np.square(np.sin(diffLat/2)) + np.cos(lat1) * np.cos(lat2) * np.square(np.sin(diffLon/2)) ) )
    
    return distance

In [ ]:
data['haversine'] = haversine_distance(data['pickup_longitude'], data['dropoff_longitude'],
                                       data['pickup_latitude'], data['dropoff_latitude'])

plt.figure(figsize = (12, 6))
for f, grouped in data.groupby('fare-bin'):
    sns.kdeplot(grouped['haversine'], label = f'{f}')

plt.xlabel('distance')
plt.ylabel('density')
plt.title('Haversine Distance by Fare Amount');

In [ ]:
data.groupby('fare-bin')['haversine'].agg(['mean', 'count'])

## Tempo de pegada do passageiro

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html

In [ ]:
data['pickup_datetime'].head()

In [ ]:
def info_from_pickup_datetime(data):
    data['year_pickup'] = data['pickup_datetime'].dt.year
    data['month_pickup'] = data['pickup_datetime'].dt.month
    data['day_pickup'] = data['pickup_datetime'].dt.day
    data['hour_pickup'] = data['pickup_datetime'].dt.hour

In [ ]:
info_from_pickup_datetime(data)

In [ ]:
data.head()

In [ ]:
data.corr()

In [ ]:
corrs = data.corr()
plt.figure(figsize=(8,8))
corrs['fare_amount'].plot.bar(color = 'b');
plt.title('Correlation with Fare Amount');

## Removendo features desnecessarias

In [ ]:
data.to_csv('data.csv', index=False)

In [ ]:
fare = data['fare-bin']
data.drop(labels=['pickup_datetime', 'fare-bin'], axis=1, inplace=True)

In [ ]:
data.head()

# Machine Learning

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)


In [ ]:
lr = LinearRegression()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(data, np.array(data['fare_amount']), 
                                                      stratify = fare,
                                                      random_state = RSEED, test_size = 0.2)

In [ ]:
lr.fit(X_train[['abs_lat_diff', 'abs_lon_diff', 'passenger_count']], y_train)

In [ ]:
def metrics(train_pred, valid_pred, y_train, y_valid):
    # Calculando mean squared error
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    valid_rmse = np.sqrt(mean_squared_error(y_valid, valid_pred))
    
    # Calculando absolute percentage error
    train_ape = abs((y_train - train_pred) / y_train)
    valid_ape = abs((y_valid - valid_pred) / y_valid)
    
    # Colocando os infinitos para zerp
    train_ape[train_ape == np.inf] = 0
    train_ape[train_ape == -np.inf] = 0
    valid_ape[valid_ape == np.inf] = 0
    valid_ape[valid_ape == -np.inf] = 0
    
    # Fazendo o percentual     
    train_mape = 100 * np.mean(train_ape)
    valid_mape = 100 * np.mean(valid_ape)
    
    return train_rmse, valid_rmse, train_mape, valid_mape

def evaluate(model, features, X_train, X_valid, y_train, y_valid):
    # Fazendo a predição com o modelo
    train_pred = model.predict(X_train[features])
    valid_pred = model.predict(X_valid[features])
    
    # Obtendo os resuktados das metricas
    train_rmse, valid_rmse, train_mape, valid_mape = metrics(train_pred, valid_pred,
                                                             y_train, y_valid)
         
    print(f'Training:   rmse = ${round(train_rmse, 2)} \t mape = {round(train_mape, 2)}%')
    print(f'Validation: rmse = ${round(valid_rmse, 2)} \t mape = {round(valid_mape, 2)}%')

In [ ]:
evaluate(lr, ['abs_lat_diff', 'abs_lon_diff', 'passenger_count'], 
        X_train, X_valid, y_train, y_valid)

In [ ]:
train_mean = y_train.mean()

# Criando uma lista com a mesma predição para todos os dados de treino e teste
train_preds = [train_mean for _ in range(len(y_train))]
valid_preds = [train_mean for _ in range(len(y_valid))]

tr, vr, tm, vm = metrics(train_preds, valid_preds, y_train, y_valid)

print(f'Baseline Training:   rmse = {round(tr, 2)} \t mape = {round(tm, 2)}')
print(f'Baseline Validation: rmse = {round(vr, 2)} \t mape = {round(vm, 2)}')

#### Agora é sua vez